# Running graphs with `tf.Session()`

The `tf.Session` class is used to launch a graph that has been previously created. In this example, our computational graph implements just a counter:



In [1]:
import tensorflow as tf
tf.reset_default_graph()

state = tf.Variable(0, name='state')
step = tf.constant(1, name='step')
increment = tf.add(state, step, name='increment')
update = tf.assign(state, increment, name='update')

Now, let's try to run our counter in a session.  
First, we create a new session object.

In [2]:
sess = tf.Session()
print(sess)

At the beginning of a session lifecycle, before running graph, you need to explicitly initialize all the variables to be used in the session. 

In [3]:
init_op = tf.global_variables_initializer()
sess.run(init_op)

Then we can run our computation.

In [4]:
print sess.run(state)
for _ in range(9):
    print sess.run(update)

0
1
2
3
4
5
6
7
8
9


A `tf.Session` can owns resources that must be released at the end of the session lifecycle.

In [5]:
sess.close()

Ok, let's put all together and use the pythonic context idiom so that we shall not care about disposing the sessio  object and releasing resources:

In [7]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print sess.run(state)
    for i in range(9):
        print sess.run(update)

0
1
2
3
4
5
6
7
8
9


...et voilà!